In [1]:
from googleapiclient.discovery import build
import pandas as pd

API_KEY = "api-key-here"

youtube = build("youtube", "v3", developerKey=API_KEY)

def get_trending_videos():
    request = youtube.videos().list(
        part="snippet,statistics",
        chart="mostPopular",
        regionCode="IN",
        maxResults=50
    )
    response = request.execute()
    
    data = []

    for item in response['items']:
        snippet = item['snippet']
        stats = item.get('statistics', {})

        data.append({
            "video_id": item["id"],
            "title": snippet["title"],
            "channel": snippet["channelTitle"],
            "published_at": snippet["publishedAt"],
            "category_id": snippet["categoryId"],
            "tags": ",".join(snippet.get("tags", [])),
            "views": int(stats.get("viewCount", 0)),
            "likes": int(stats.get("likeCount", 0)),
            "comments": int(stats.get("commentCount", 0))
        })

    return pd.DataFrame(data)


df = get_trending_videos()
df.head()

,video_id,title,channel,published_at,category_id,tags,views,likes,comments
0,YyepU5ztLf4,"Shararat | Dhurandhar | Ranveer, Aditya Dhar, ...",Saregama Music,2025-12-09T09:33:03Z,10,"dhurandhar song,dhurandhar song balochi,dhuran...",8213353,138951,2970
1,ZEiFcHGJYzA,Mark - Official Trailer | Kichcha Sudeepa | Vi...,Saregama Kannada,2025-12-07T06:30:41Z,1,"Mark Kannada,Mark Kannada Trailer,Mark Officia...",10258945,421542,21546
2,dgrSdn2qOxI,CARRY is GONE FOREVER in Minecraft!🥲,Carry Depie,2025-12-10T09:17:40Z,20,"carry depie,minecraft",642585,11230,558
3,hXvGfWpkuqc,Dekhlenge Saala Song Promo | Ustaad Bhagat Sin...,Sony Music South,2025-12-09T12:58:07Z,10,"Sony Music South,Sony Music,Latest Song,Tamil ...",851723,86002,1945
4,lLWTiFZqqXg,"One Two Cha Cha Chaa Teaser|Ashutosh, Lalit, A...",Divo Music Hindi,2025-12-08T08:34:00Z,10,"divo music,divo music label,divo music hindi,h...",1721971,34353,2798


In [2]:
category_map = {
    "1": "Film & Animation",
    "2": "Autos & Vehicles",
    "10": "Music",
    "15": "Pets & Animals",
    "17": "Sports",
    "19": "Travel & Events",
    "20": "Gaming",
    "22": "People & Blogs",
    "23": "Comedy",
    "24": "Entertainment",
    "25": "News & Politics",
    "26": "Howto & Style",
    "27": "Education",
    "28": "Science & Technology"
}

df["category"] = df["category_id"].map(category_map)

In [4]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.1 MB/s eta 0:00:00


In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

df["sentiment"] = df["title"].apply(lambda x:
    analyzer.polarity_scores(x)["compound"]
)

df["sentiment_label"] = df["sentiment"].apply(
    lambda x: "positive" if x > 0.05 else
              "negative" if x < -0.05 else "neutral"
)

In [6]:
df = df.drop_duplicates(subset="video_id")

df["published_at"] = pd.to_datetime(df["published_at"])

# sort by views
df = df.sort_values("views", ascending=False)

In [7]:
df.to_csv("indian_youtube_trending_2024_25.csv", index=False)